In [ ]:
# ==============================
# CROP RECOMMENDATION MODEL
# ==============================

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

# ---------------------------------
# 1. Load Dataset
# ---------------------------------
df = pd.read_csv("Crop_recommendation.csv")

X = df.drop("label", axis=1)
y = df["label"]

# ---------------------------------
# 2. Train-Test Split
# ---------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ---------------------------------
# 3. Feature Scaling
# ---------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ---------------------------------
# 4. Model + Hyperparameter Tuning
# ---------------------------------
rf = RandomForestClassifier()

params = {
    "n_estimators": [200, 300],
    "max_depth": [10, 20, None],
    "min_samples_split": [2, 5],
}

grid = GridSearchCV(
    rf,
    params,
    cv=5,
    n_jobs=-1,
    scoring="accuracy"
)

grid.fit(X_train_scaled, y_train)

best_model = grid.best_estimator_
print("Best Parameters:", grid.best_params_)

# ---------------------------------
# 5. Evaluate Model
# ---------------------------------
preds = best_model.predict(X_test_scaled)

print("\nAccuracy:", accuracy_score(y_test, preds))
print("\nClassification Report:\n", classification_report(y_test, preds))

# ---------------------------------
# 6. Save Model for Later Use
# ---------------------------------
with open("crop_recommendation_model.pkl", "wb") as f:
    pickle.dump({"model": best_model, "scaler": scaler}, f)

print("\nModel saved as crop_model.pkl")


Best Parameters: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 200}

Accuracy: 0.9931818181818182

Classification Report:
               precision    recall  f1-score   support

       apple       1.00      1.00      1.00        23
      banana       1.00      1.00      1.00        21
   blackgram       1.00      1.00      1.00        20
    chickpea       1.00      1.00      1.00        26
     coconut       1.00      1.00      1.00        27
      coffee       1.00      1.00      1.00        17
      cotton       1.00      1.00      1.00        17
      grapes       1.00      1.00      1.00        14
        jute       0.92      1.00      0.96        23
 kidneybeans       1.00      1.00      1.00        20
      lentil       0.92      1.00      0.96        11
       maize       1.00      1.00      1.00        21
       mango       1.00      1.00      1.00        19
   mothbeans       1.00      0.96      0.98        24
    mungbean       1.00      1.00      1.00        19


In [ ]:
# =======================================================
# STACKING ENSEMBLE FOR CROP RECOMMENDATION
# =======================================================

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
import pickle

# OPTIONAL: Install XGBoost if not installed
# pip install xgboost
from xgboost import XGBClassifier

# --------------------------------------------------
# 1. Load Data
# --------------------------------------------------
df = pd.read_csv("Crop_recommendation.csv")

X = df.drop("label", axis=1)
y = df["label"]

# --------------------------------------------------
# 2. Train-Test Split
# --------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# --------------------------------------------------
# 3. Scale Features
# --------------------------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --------------------------------------------------
# 4. Base Models
# --------------------------------------------------
base_models = [
    ('rf', RandomForestClassifier(
        n_estimators=300,
        max_depth=20,
        random_state=42
    )),

    ('xgb', XGBClassifier(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=8,
        subsample=0.9,
        colsample_bytree=0.9,
        eval_metric='mlogloss'
    )),

    ('svc', SVC(
        kernel='rbf',
        probability=True,
        C=5,
        gamma='scale'
    ))
]

# --------------------------------------------------
# 5. Meta-Learner
# --------------------------------------------------
meta_model = LogisticRegression(max_iter=200)

# --------------------------------------------------
# 6. Stacking Classifier
# --------------------------------------------------
stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    stack_method='predict_proba',
    n_jobs=-1
)

# --------------------------------------------------
# 7. Train the Model
# --------------------------------------------------
stack_model.fit(X_train_scaled, y_train)

# --------------------------------------------------
# 8. Evaluate the Ensemble
# --------------------------------------------------
preds = stack_model.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, preds))
print("\nClassification Report:\n", classification_report(y_test, preds))

# --------------------------------------------------
# 9. Save the Model
# --------------------------------------------------
with open("ensemble_crop_model.pkl", "wb") as f:
    pickle.dump({
        "model": stack_model,
        "scaler": scaler
    }, f)

print("\nModel saved as ensemble_crop_model.pkl")


Accuracy: 0.990909090909091

Classification Report:
               precision    recall  f1-score   support

       apple       1.00      1.00      1.00        23
      banana       1.00      1.00      1.00        21
   blackgram       1.00      1.00      1.00        20
    chickpea       1.00      1.00      1.00        26
     coconut       1.00      1.00      1.00        27
      coffee       1.00      1.00      1.00        17
      cotton       1.00      1.00      1.00        17
      grapes       1.00      1.00      1.00        14
        jute       0.92      0.96      0.94        23
 kidneybeans       1.00      1.00      1.00        20
      lentil       0.92      1.00      0.96        11
       maize       1.00      1.00      1.00        21
       mango       1.00      1.00      1.00        19
   mothbeans       1.00      0.96      0.98        24
    mungbean       1.00      1.00      1.00        19
   muskmelon       1.00      1.00      1.00        17
      orange       1.00     